In [1]:
from jina import Document, DocumentArray
import pandas as pd 
import random

In [2]:
def prep_docs(input_file : str, num_size = -1, shuffle=True):
    docs = DocumentArray()
    error = []
    print(f"Processing {input_file}")

    df = pd.read_csv(input_file)
    df = df.dropna().drop_duplicates()
    n = df.shape[0]
    

    if shuffle:
        df = df.sample(frac = 1)
    
    num_size = n if num_size == -1 else num_size

    for idx in range(n):
        plyr = df.iloc[idx, 3]
        psong = df.iloc[idx, 2]
        psinger = df.iloc[idx, 1]
        doctext = plyr
        if len(doctext) > 0:
            doc = Document(text=doctext)
            doc.tags['song_name'] = psong
            doc.tags['singer'] = psinger
            docs.extend([doc])
            doc = Document(text = plyr)

    return docs[:num_size]

In [3]:
docs = prep_docs(input_file= "lyrics-JINA - Sheet1.csv", num_size = 30, shuffle=True)
docs

Processing lyrics-JINA - Sheet1.csv


<jina.types.arrays.document.DocumentArray (length=30) at 140279612697952>

In [4]:
#doc.tags['song_name'] = song_name


In [5]:
len(docs)

30

In [6]:
docs[0].text, docs[0].tags['song_name']

('Ay\nFonsi\nDY\nOh-oh\nOh no, oh no (oh)\nHey yeah\nDiridiri, dirididi Daddy\nGo\nSí, sabes que ya llevo un rato mirándote\nTengo que bailar contigo hoy (DY)\nVi que tu mirada ya estaba llamándome\nMuéstrame el camino que yo voy\nOh\nTú, tú eres el imán y yo soy el metal\nMe voy acercando y voy armando el plan\nSolo con pensarlo se acelera el pulso\nOh yeah\nYa, ya me está gustando más de lo normal\nTodos mis sentidos van pidiendo más\nEsto hay que tomarlo sin ningún apuro\nDespacito\nQuiero respirar tu cuello despacito\nDeja que te diga cosas al oído\nPara que te acuerdes si no estás conmigo\nDespacito\nQuiero desnudarte a besos despacito\nFirmar las paredes de tu laberinto\nY hacer de tu cuerpo todo un manuscrito (sube, sube, sube)\n(Sube, sube) Oh\nQuiero ver bailar tu pelo\nQuiero ser tu ritmo (eh-oh) (uh-oh, uh-oh)\nQue le enseñes a mi boca (eh-oh) (uh-oh, uh-oh)\nTus lugares favoritos (eh-oh) (favoritos, favoritos baby)\nDéjame sobrepasar\nTus zonas de peligro (eh-oh) (uh-oh, uh

In [7]:
for i in docs:
    if len(i.text) == 0:
        print(i.text)

In [8]:
model = "sentence-transformers/paraphrase-distilroberta-base-v1" # Any model from Huggingface

In [9]:
from jina import Flow

In [10]:
flow = (
    Flow(install_requirements = True)
    .add(
        name="lyrics_text_encoder",
        uses="jinahub://TransformerTorchEncoder",
        uses_with={"pretrained_model_name_or_path": model},
    )
    .add(
        name="lyrics_text_indexer",
        uses='jinahub://SimpleIndexer',
    )
)

In [11]:
flow

In [12]:
!rm -rf workspace # Remove workspace in case we've indexed stuff before

In [13]:
with flow:
    flow.index(
        inputs=docs,
  )

⠋ 0/3 waiting lyrics_text_encoder lyrics_text_indexer gateway to be ready...      

⠧ 2/3 waiting lyrics_text_encoder to be ready...                                  

           Flow@24987[I]:🎉 Flow is ready to use!                                          
	🔗 Protocol: 		GRPC
	🏠 Local access:	0.0.0.0:54953
	🔒 Private network:	192.168.43.107:54953


In [15]:
# dir(flow)
flow.inspect()

In [16]:
docs[0].text

'Ay\nFonsi\nDY\nOh-oh\nOh no, oh no (oh)\nHey yeah\nDiridiri, dirididi Daddy\nGo\nSí, sabes que ya llevo un rato mirándote\nTengo que bailar contigo hoy (DY)\nVi que tu mirada ya estaba llamándome\nMuéstrame el camino que yo voy\nOh\nTú, tú eres el imán y yo soy el metal\nMe voy acercando y voy armando el plan\nSolo con pensarlo se acelera el pulso\nOh yeah\nYa, ya me está gustando más de lo normal\nTodos mis sentidos van pidiendo más\nEsto hay que tomarlo sin ningún apuro\nDespacito\nQuiero respirar tu cuello despacito\nDeja que te diga cosas al oído\nPara que te acuerdes si no estás conmigo\nDespacito\nQuiero desnudarte a besos despacito\nFirmar las paredes de tu laberinto\nY hacer de tu cuerpo todo un manuscrito (sube, sube, sube)\n(Sube, sube) Oh\nQuiero ver bailar tu pelo\nQuiero ser tu ritmo (eh-oh) (uh-oh, uh-oh)\nQue le enseñes a mi boca (eh-oh) (uh-oh, uh-oh)\nTus lugares favoritos (eh-oh) (favoritos, favoritos baby)\nDéjame sobrepasar\nTus zonas de peligro (eh-oh) (uh-oh, uh-

In [17]:
query_doc = Document(text= 'good')

In [18]:
with flow:
    response = flow.search(inputs=query_doc, return_results=True)

⠋ 0/3 waiting lyrics_text_encoder lyrics_text_indexer gateway to be ready...      

⠦ 2/3 waiting lyrics_text_encoder to be ready...                                  

           Flow@24987[I]:🎉 Flow is ready to use!                                          
	🔗 Protocol: 		GRPC
	🏠 Local access:	0.0.0.0:43503
	🔒 Private network:	192.168.43.107:43503


In [54]:
matches = response[0].docs[0].matches

In [55]:

matches

<jina.types.arrays.match.MatchArray (length=20) at 140458818864416>

In [59]:
for ind, i in enumerate(matches):
    print(f' song name : {ind} | lyrics : {i.tags["song_name"]} '.center(60,'='))
    print(i.text)
    print()

============ song name : 0 | lyrics : Good life ============
[Kehlani:]
Raise a cup up for all my day ones
Two middle fingers for the haters
Life's only getting greater
Straight up from nothing we go
Higher than the highest skyscraper
No Little League, we major
The proof is in the paper

[Kehlani (G-Eazy):]
We put the good in the good in the good life
We put the good in the good in the good life
We put the bad in the past, now we alright
(Eazy)
Hey, hey, hey
(Kehlani, I got you)
Hey, hey

[G-Eazy:]
And it's a feeling that I can't explain
How you make it and your team still stay the same
Stay down from the jump and they never change
Man, this a moment I could never trade, yeah
I told my moms not to stress no more
Go hit the Bentley store and no credit card debts no more
(Love you mom)
I bought the crib and it's in escrow now
So you don't ever have to worry about how you gon' pay rent no more

I put my team in position, now they makin' a killin'
Stackin' blue faces straight to the ceilin